In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thymoma"
cohort = "GSE42977"

# Input paths
in_trait_dir = "../../input/GEO/Thymoma"
in_cohort_dir = "../../input/GEO/Thymoma/GSE42977"

# Output paths
out_data_file = "../../output/preprocess/Thymoma/GSE42977.csv"
out_gene_data_file = "../../output/preprocess/Thymoma/gene_data/GSE42977.csv"
out_clinical_data_file = "../../output/preprocess/Thymoma/clinical_data/GSE42977.csv"
json_path = "../../output/preprocess/Thymoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE42977_family.soft.gz', 'GSE42977_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Thymoma/GSE42977/GSE42977_family.soft.gz
Matrix file: ../../input/GEO/Thymoma/GSE42977/GSE42977_series_matrix.txt.gz


Background Information:
!Series_title	"Sequential Binary Gene-Ratio Tests Define a Novel Molecular Diagnostic Strategy for Malignant Pleural Mesothelioma"
!Series_summary	"The gene-expression ratio  technique was used to design a molecular signature to diagnose MPM from among other potentially confounding diagnoses and differentiate the epithelioid from the sarcomatoid histological subtype of MPM."
!Series_overall_design	"Microarray analysis was performed on 113 specimens including MPMs and a spectrum of tumors and benign tissues comprising the differential diagnosis of MPM.  A sequential combination of binary gene-expression ratio tests was developed to discriminate MPM from other thoracic malignancies .  This method was compared to other bioinformatic tools and this signature was validated  in an independent set of 170 samples.  Functional enrichment analysis was performed to identify differentially expressed probes."
Sample Characteristics Dictionary:
{0: ['tissue: control', 'tissue

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any, Union
import numpy as np

# GEO matrix files require special handling due to their format
# First, identify where the actual data begins (after the metadata)
data_start_line = 0
with gzip.open(f"{in_cohort_dir}/GSE42977_series_matrix.txt.gz", 'rt') as file:
    for i, line in enumerate(file):
        if line.startswith('!Sample_'):
            data_start_line = i
            break

# Read the sample characteristics table from the matrix file
sample_data_lines = []
with gzip.open(f"{in_cohort_dir}/GSE42977_series_matrix.txt.gz", 'rt') as file:
    for i, line in enumerate(file):
        if line.startswith('!Sample_characteristics_ch1'):
            sample_data_lines.append(line.strip())

# Process the sample characteristics to create a dataframe
clinical_data = {}
for i, line in enumerate(sample_data_lines):
    # Extract the values after "!Sample_characteristics_ch1\t"
    values = line.replace('!Sample_characteristics_ch1\t', '').split('\t')
    if i == 0:  # First line - initialize the dictionary with column names
        for j, val in enumerate(values):
            clinical_data[f'Sample_{j}'] = []
    
    # Add values to the dictionary
    for j, val in enumerate(values):
        if j < len(clinical_data):  # Ensure we don't exceed the initialized columns
            clinical_data[f'Sample_{j}'].append(val)

# Convert to DataFrame and transpose to get samples as rows
clinical_df = pd.DataFrame(clinical_data).transpose()

# 1. Gene Expression Data Availability
# Based on the series title and overall design, this appears to be a microarray gene expression dataset
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Thymoma), we can see 'tissue: Thymoma' and 'tissue: Metastatic Thymoma' in the characteristics
# at index 0, so we can use this to identify Thymoma cases
trait_row = 0  # First row contains tissue type information

# Age is not mentioned in the sample characteristics
age_row = None

# Gender is not mentioned in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> Union[int, None]:
    """
    Convert tissue type to binary (1 for Thymoma, 0 for other)
    """
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value contains "Thymoma"
    if "Thymoma" in value:
        return 1
    else:
        return 0

def convert_age(value: str) -> Union[float, None]:
    """
    Convert age to continuous numeric value
    Not used in this dataset but included for completeness
    """
    return None

def convert_gender(value: str) -> Union[int, None]:
    """
    Convert gender to binary (0 for female, 1 for male)
    Not used in this dataset but included for completeness
    """
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We need to implement our own get_feature_data function since it's not provided in the library
    def get_feature_data(clinical_df, row_idx, feature_name, convert_func):
        row_data = clinical_df.iloc[row_idx]
        result = {}
        for col in clinical_df.columns:
            result[col] = convert_func(row_data[col])
        return pd.DataFrame({feature_name: result}).T
    
    # Use the function to create selected clinical features
    feature_list = []
    
    # Add trait data
    trait_data = get_feature_data(clinical_df, trait_row, trait, convert_trait)
    feature_list.append(trait_data)
    
    # Add age data if available
    if age_row is not None:
        age_data = get_feature_data(clinical_df, age_row, 'Age', convert_age)
        feature_list.append(age_data)
    
    # Add gender data if available
    if gender_row is not None:
        gender_data = get_feature_data(clinical_df, gender_row, 'Gender', convert_gender)
        feature_list.append(gender_data)
    
    # Combine all features
    selected_clinical_df = pd.concat(feature_list, axis=0)
    
    # Preview the extracted data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{0: [0]}
Clinical data saved to ../../output/preprocess/Thymoma/clinical_data/GSE42977.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 46713
First 20 gene/probe identifiers:
Index(['ILMN_10000', 'ILMN_100000', 'ILMN_100007', 'ILMN_100009', 'ILMN_10001',
       'ILMN_100010', 'ILMN_10002', 'ILMN_100028', 'ILMN_100030',
       'ILMN_100031', 'ILMN_100034', 'ILMN_100037', 'ILMN_10004', 'ILMN_10005',
       'ILMN_100054', 'ILMN_100059', 'ILMN_10006', 'ILMN_100075',
       'ILMN_100079', 'ILMN_100083'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# After reviewing the gene identifiers, I can conclude they are not human gene symbols
# These are Illumina BeadChip probe IDs (ILMN_prefix) which need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'GB_ACC': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'SEQUENCE': ['CTCTCTAAAGGGACAACAGAGTGGACAGTCAAGGAACTCCACATATTCAT', 'GGGGTCAAGCCCAGGTGAAATGTGGATTGGAAAAGTGCTTCCCTTGCCCC', 'CCAGGCTGTAAAAGCAAAACCTCGTATCAGCTCTGGAACAATACCTGCAG', 'CCAGACAGGAAGCATCAAGCCCTTCAGGAAAGAATATGCGAGAGTGCTGC', 'TGTGCAGAAAGCTGATGGAAGGGAGAAAGAATGGAAGTGGGTCACACAGC'], 'Definition': ['UI-CF-EC0-abi-c-12-0-UI.s1 UI-CF-EC0 Homo sapiens cDNA clone UI-CF-EC0-abi-c-12-0-UI 3, mRNA sequence', 'PREDICTED: Homo sapiens similar to spectrin domain with coiled-coils 1 (LOC441782), mRNA.', 'Homo sapiens jumonji domain containing 1A (JMJD1A), mRNA.', 'hi56g05.x1 Soares_NFL_T_GBC_S1 Homo sapiens cDNA clone IMAGE:2976344 3, mRNA sequence', 'wk77d04.x1 NCI_CGAP_Pan1 Homo sapiens cDNA clone IMAGE:2421415 3, mRNA sequence'], 'Ontology': [nan, nan, nan, nan, na

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine the gene identifiers in both datasets
# From previous output, gene expression data has "ILMN_" IDs as the index
# From annotation preview, we can see that "ID" column contains the same ILMN identifiers
# And the "Symbol" column contains gene symbols we want to map to

print("Identifying probe-to-gene mapping columns:")
print(f"Probe ID column: 'ID'")
print(f"Gene symbol column: 'Symbol'")

# 2. Get gene mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First 5 rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# Using the apply_gene_mapping function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])

# Preview some of the gene expression values
print("\nPreview of gene expression values for first 3 genes and first 3 samples:")
sample_cols = gene_data.columns[:3]
print(gene_data.iloc[:3, :3])

# Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Mapped gene expression data saved to: {out_gene_data_file}")


Identifying probe-to-gene mapping columns:
Probe ID column: 'ID'
Gene symbol column: 'Symbol'


Gene mapping dataframe shape: (28566, 2)
First 5 rows of gene mapping:
           ID       Gene
1  ILMN_35826  LOC441782
2  ILMN_25544     JMJD1A
6  ILMN_10519      NCOA3
8  ILMN_17234  LOC389834
9  ILMN_19244   C17orf77
Gene expression data shape after mapping: (18401, 117)
First 5 gene symbols after mapping:
Index(['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A3GALT2'], dtype='object', name='Gene')

Preview of gene expression values for first 3 genes and first 3 samples:
       GSM1054230  GSM1054231  GSM1054232
Gene                                     
A1BG     0.903276    0.821580    0.865428
A2BP1    1.719921    8.432093    1.797852
A2M     27.154642   17.070914   34.490890


Mapped gene expression data saved to: ../../output/preprocess/Thymoma/gene_data/GSE42977.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene data shape: {gene_data.shape}")
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to: {out_gene_data_file}")

# 2. Process the clinical data based on the available information
# First check what clinical_data actually is
print("Type of clinical_data:", type(clinical_data))

# We need to extract clinical data again with the correct function
try:
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Verify that clinical_data is a DataFrame
    print("Type of clinical_data after extraction:", type(clinical_data))
    
    # Define the convert_trait function specific to this dataset
    def convert_trait(value):
        """Convert tissue type to binary (1 for Thymoma, 0 for other)"""
        if not isinstance(value, str):
            return None
        
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Look for Thymoma in the value
        if 'Thymoma' in value:
            return 1
        else:
            return 0
    
    # Extract clinical features using only the available row
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=0,  # Only row with tissue information
        convert_trait=convert_trait,
        age_row=None,  # Age not available
        convert_age=None,
        gender_row=None,  # Gender not available
        convert_gender=None
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, trait)
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset from GSE42977 focused on multiple tissue types including Thymoma. Only tissue information was available for clinical features."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    print(f"Full error: {type(e).__name__}: {str(e)}")
    
    # Create a minimal DataFrame for validation
    empty_df = pd.DataFrame({trait: []})
    
    # If clinical data processing fails, update the cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=False,  # Set a value for validation
        df=empty_df,
        note="Failed to extract clinical data. Gene expression data is available but clinical feature extraction failed."
    )
    print("Dataset validation failed due to missing or corrupted clinical data. Only gene data saved.")

Original gene data shape: (18401, 117)
Normalized gene data shape: (17207, 117)


Normalized gene expression data saved to: ../../output/preprocess/Thymoma/gene_data/GSE42977.csv
Type of clinical_data: <class 'dict'>


Type of clinical_data after extraction: <class 'pandas.core.frame.DataFrame'>
Clinical data preview:
{'GSM1054230': [0.0], 'GSM1054231': [0.0], 'GSM1054232': [0.0], 'GSM1054233': [0.0], 'GSM1054234': [0.0], 'GSM1054235': [0.0], 'GSM1054236': [0.0], 'GSM1054237': [0.0], 'GSM1054238': [0.0], 'GSM1054239': [0.0], 'GSM1054240': [0.0], 'GSM1054241': [0.0], 'GSM1054242': [0.0], 'GSM1054243': [1.0], 'GSM1054244': [0.0], 'GSM1054245': [0.0], 'GSM1054246': [0.0], 'GSM1054247': [0.0], 'GSM1054248': [0.0], 'GSM1054249': [0.0], 'GSM1054250': [0.0], 'GSM1054251': [0.0], 'GSM1054252': [0.0], 'GSM1054253': [0.0], 'GSM1054254': [0.0], 'GSM1054255': [0.0], 'GSM1054256': [1.0], 'GSM1054257': [0.0], 'GSM1054258': [0.0], 'GSM1054259': [0.0], 'GSM1054260': [0.0], 'GSM1054261': [0.0], 'GSM1054262': [0.0], 'GSM1054263': [0.0], 'GSM1054264': [0.0], 'GSM1054265': [1.0], 'GSM1054266': [0.0], 'GSM1054267': [0.0], 'GSM1054268': [0.0], 'GSM1054269': [0.0], 'GSM1054270': [0.0], 'GSM1054271': [1.0], 'GSM1054272': [0

Shape of linked data after handling missing values: (117, 17208)
For the feature 'Thymoma', the least common label is '1.0' with 6 occurrences. This represents 5.13% of the dataset.
The distribution of the feature 'Thymoma' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Thymoma/GSE42977.csv
